In [1]:
import pandas as pd

### Extraccion

In [2]:
df = pd.read_json('Datasets\\steam_games.json.gz', lines=True)
df.head()

,publisher,genres,app_name,title,url,release_date,tags,reviews_url,specs,price,early_access,id,developer
0,None,None,None,None,None,None,None,None,None,None,NaN,NaN,None
1,None,None,None,None,None,None,None,None,None,None,NaN,NaN,None
2,None,None,None,None,None,None,None,None,None,None,NaN,NaN,None
3,None,None,None,None,None,None,None,None,None,None,NaN,NaN,None
4,None,None,None,None,None,None,None,None,None,None,NaN,NaN,None


In [135]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 120445 entries, 0 to 120444
Data columns (total 13 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   publisher     24083 non-null  object 
 1   genres        28852 non-null  object 
 2   app_name      32133 non-null  object 
 3   title         30085 non-null  object 
 4   url           32135 non-null  object 
 5   release_date  30068 non-null  object 
 6   tags          31972 non-null  object 
 7   reviews_url   32133 non-null  object 
 8   specs         31465 non-null  object 
 9   price         30758 non-null  object 
 10  early_access  32135 non-null  float64
 11  id            32133 non-null  float64
 12  developer     28836 non-null  object 
dtypes: float64(2), object(11)
memory usage: 11.9+ MB


In [313]:
df.shape

(120445, 13)

## Transformacion

#### Eliminacion de nulos
Verificamos la cantidad de valores nulos de cada columna

In [3]:
# Cuenta la cantidad de nulos por columna
print(df.isnull().sum())
print("Total de valores nulos:",df.isnull().sum().sum())

publisher       96362
genres          91593
app_name        88312
title           90360
url             88310
release_date    90377
tags            88473
reviews_url     88312
specs           88980
price           89687
early_access    88310
id              88312
developer       91609
dtype: int64
Total de valores nulos: 1168997


Se puede observar que la cantidad de valores nulos es muy alta. Procederemos a eliminar solo aquellas filas donde todas las columnas contengan valores nulos

In [4]:
# con how='all' eliminamos solo aquellas filas que estan completamente vacias
df.dropna(how='all', inplace=True)

Verificamos cuantos valores nulos quedaron

In [5]:
print(df.isnull().sum())

publisher       8052
genres          3283
app_name           2
title           2050
url                0
release_date    2067
tags             163
reviews_url        2
specs            670
price           1377
early_access       0
id                 2
developer       3299
dtype: int64


#### Eliminacion de duplicados

verificamos a partir de 'id' si hay datos duplicados

In [6]:
df['id'].duplicated().sum()

2

Vemos que hay valores duplicados. Para obtenerlos se hace lo siguiente:

In [7]:
# Cuento las veces que aparece cada id
counts_id = df['id'].value_counts()
# Imprime los que aparecen mas de 1 vez
print(counts_id[counts_id>1])

id
612880.0    2
Name: count, dtype: int64


In [8]:
duplicado = df[df['id'] == 612880.0]
duplicado

,publisher,genres,app_name,title,url,release_date,tags,reviews_url,specs,price,early_access,id,developer
102204,Bethesda Softworks,[Action],Wolfenstein II: The New Colossus,Wolfenstein II: The New Colossus,http://store.steampowered.com/app/612880/,2017-10-26,"[Action, FPS, Gore, Violent, Alternate History...",http://steamcommunity.com/app/612880/reviews/?...,"[Single-player, Steam Achievements, Full contr...",59.99,0.0,612880.0,Machine Games
102883,Bethesda Softworks,[Action],Wolfenstein II: The New Colossus,Wolfenstein II: The New Colossus,http://store.steampowered.com/app/612880/Wolfe...,2017-10-26,"[Action, FPS, Gore, Violent, Alternate History...",http://steamcommunity.com/app/612880/reviews/?...,"[Single-player, Steam Achievements, Full contr...",59.99,0.0,612880.0,Machine Games


In [9]:
# Elimino uno de los duplicados a partir del indice de la fila
df.drop([102883],inplace=True)

#### Eliminacion de columnas
Se puede observar que 'app_name' y 'title' tienen valores similares, por lo tanto se eliminara la columna 'title' ya que tiene mas cantidad de valores nulos.

Se eliminan las columnas 'url' , 'reviews_url' , 'early_access', 'specs'  ya que no se necesitan.

In [10]:
df.drop(['title'],axis=1,inplace=True) # La dejo aparte porque no estoy seguro
df.drop(['url','reviews_url','early_access','specs'],axis=1,inplace=True)

In [11]:
df.head(3)

,publisher,genres,app_name,release_date,tags,price,id,developer
88310,Kotoshiro,"[Action, Casual, Indie, Simulation, Strategy]",Lost Summoner Kitty,2018-01-04,"[Strategy, Action, Indie, Casual, Simulation]",4.99,761140.0,Kotoshiro
88311,"Making Fun, Inc.","[Free to Play, Indie, RPG, Strategy]",Ironbound,2018-01-04,"[Free to Play, Strategy, Indie, RPG, Card Game...",Free To Play,643980.0,Secret Level SRL
88312,Poolians.com,"[Casual, Free to Play, Indie, Simulation, Sports]",Real Pool 3D - Poolians,2017-07-24,"[Free to Play, Simulation, Sports, Casual, Ind...",Free to Play,670290.0,Poolians.com


#### Cambio de tipo de datos

Remplazo el tipo de dato 'id' de float a string y normalizo los datos

In [12]:
df['id'] = df['id'].astype(str)

# Remplazo los '.0' por '' (vacio)
df['id'] = df['id'].str.replace('.0','')
# Elimino lo vacio
df['id'] = df['id'].str.strip()

Cambio el tipo de dato 'release_date' de string a datetime

In [13]:
# Este cambio se hace para luego extraer el año
df['release_date']= pd.to_datetime(df['release_date'], errors='coerce')

In [14]:
# Extraigo el año lo almaceno como string en 'release_year' y relleno con 'sin dato' los nulos
df['release_year'] = df['release_date'].dt.year.fillna('sin dato').astype(str)

# Esto va porque al pasar el año a string aparece con .0 al final (ej: 2015.0)
df['release_year'] = df['release_year'].str.replace('.0','')
#Elimina los espacios en blanco 
df['release_year'] = df['release_year'].str.strip()

In [15]:
#Elimino la columna 'release_date'
df.drop(['release_date'],axis=1,inplace=True)

Cambio el tipo de dato de 'price' de string a float

In [16]:
# Visualizo los valores de la columna price
precios = df['price'].unique()
precios

array([4.99, 'Free To Play', 'Free to Play', 0.99, 2.99, 3.99, 9.99,
       18.99, 29.99, None, 'Free', 10.99, 1.5899999999999999, 14.99, 1.99,
       59.99, 8.99, 6.99, 7.99, 39.99, 19.99, 7.49, 12.99, 5.99, 2.49,
       15.99, 1.25, 24.99, 17.99, 61.99, 3.49, 11.99, 13.99, 'Free Demo',
       'Play for Free!', 34.99, 74.76, 1.49, 32.99, 99.99, 14.95, 69.99,
       16.99, 79.99, 49.99, 5.0, 44.99, 13.98, 29.96, 119.99, 109.99,
       149.99, 771.71, 'Install Now', 21.99, 89.99,
       'Play WARMACHINE: Tactics Demo', 0.98, 139.92, 4.29, 64.99,
       'Free Mod', 54.99, 74.99, 'Install Theme', 0.89, 'Third-party',
       0.5, 'Play Now', 299.99, 1.29, 3.0, 15.0, 5.49, 23.99, 49.0, 20.99,
       10.93, 1.3900000000000001, 'Free HITMAN™ Holiday Pack', 36.99,
       4.49, 2.0, 4.0, 9.0, 234.99, 1.9500000000000002, 1.5, 199.0, 189.0,
       6.66, 27.99, 10.49, 129.99, 179.0, 26.99, 399.99, 31.99, 399.0,
       20.0, 40.0, 3.33, 199.99, 22.99, 320.0, 38.85, 71.7, 59.95, 995.0,
       27.49,

Para poder llevar estos datos a float, se implementa una funcion que toma como parametro el precio y en caso de ser un numero lo retorna como float, si no es un numero convierte el valor a 0

In [17]:
# Se utiliza (try except) para poder controlar los errores (toma una desicion u otra dependiendo si hay un error o no)
# y de esta forma no obtenemos errores en consola
def conversion(precio):
    try:    
        return float(precio)
    except(ValueError, TypeError):
        return float(0.0)
    
df['price'] = df['price'].apply(conversion)


In [18]:
df.head(3)

,publisher,genres,app_name,tags,price,id,developer,release_year
88310,Kotoshiro,"[Action, Casual, Indie, Simulation, Strategy]",Lost Summoner Kitty,"[Strategy, Action, Indie, Casual, Simulation]",4.99,761140,Kotoshiro,2018
88311,"Making Fun, Inc.","[Free to Play, Indie, RPG, Strategy]",Ironbound,"[Free to Play, Strategy, Indie, RPG, Card Game...",0.00,643980,Secret Level SRL,2018
88312,Poolians.com,"[Casual, Free to Play, Indie, Simulation, Sports]",Real Pool 3D - Poolians,"[Free to Play, Simulation, Sports, Casual, Ind...",0.00,670290,Poolians.com,2017


#### Transformacion de columna 'genres'

Se observa que varias filas de la columna 'genres' contienen listas. Para transformar esto a String, se implementa una funcion donde se obtiene el primer elemento de cada una de las listas y se asigna a cada fila

In [19]:
# Funcion para obtener  
def clean_genres(genres):
    if isinstance(genres, list):# Si 'genres' es una lista. Devuelve el primer elemento
        return genres[0]
    elif isinstance(genres, str): # Si 'genres' es un string. Devuelve ese valor
        return genres
    else:
        return 'Pending classification' 

# Se aplica la funcion a la columna 'genres'  
df['genres'] = df['genres'].apply(clean_genres)

Cuento la cantidad de valores que quedaron para cada genero

In [20]:
df['genres'].value_counts()

genres
Action                       11317
Casual                        4340
Adventure                     4328
Indie                         3299
Pending classification        3283
Simulation                    1852
Strategy                      1143
RPG                            685
Free to Play                   523
Racing                         454
Design &amp; Illustration      327
Animation &amp; Modeling       179
Utilities                       93
Audio Production                84
Sports                          63
Massively Multiplayer           49
Education                       46
Video Production                21
Software Training               21
Web Publishing                   8
Photo Editing                    8
Accounting                       7
Early Access                     4
Name: count, dtype: int64

Se puede observar que el valor asignado como 'Pending Classification' tiene una gran cantidad de valores. Como la columna 'tags' tiene valores similares a 'genres', se va a crear una funcion para reemplazar los 'Pending Classification' por valores de 'tags'

In [21]:
# Esta funcion remplaza 'Pending classification' por el primer valor de cada fila de la columna 'tags'
def fill_rows(tags,genres):
    if genres == 'Pending classification':
        
        if isinstance(tags,list): # Si en tags hay una lista, devuelve el primer valor
            return tags[0]
            
        elif isinstance(tags, str): # Si 'tags' es un string. Devuelve ese valor
             return tags
         
    return genres
# Se crea una nueva columna y se le aplica la funcion
df['genres_clean']= df.apply(lambda row: fill_rows(row['tags'], row['genres']), axis=1)

Luego de aplicar la funcion los 'Pending classification' se redujeron de 3283 a 139

In [22]:
print((df['genres_clean'] == 'Pending classification').sum())

139


Elimino las columnas 'genres' y 'tags', ya que no les daremos utilidad

In [23]:

df.drop(['genres','tags'],axis=1,inplace=True)

In [24]:
# Muevo la columna 'genres_clean' (6) a la segunda posicion
df = df.iloc[:,[0,6,1,2,3,4,5]]
df

,publisher,genres_clean,app_name,price,id,developer,release_year
88310,Kotoshiro,Action,Lost Summoner Kitty,4.99,761140,Kotoshiro,2018
88311,"Making Fun, Inc.",Free to Play,Ironbound,0.00,643980,Secret Level SRL,2018
88312,Poolians.com,Casual,Real Pool 3D - Poolians,0.00,670290,Poolians.com,2017
88313,彼岸领域,Action,弹炸人2222,0.99,767400,彼岸领域,2017
88314,None,Action,Log Challenge,2.99,773570,None,sin dato
...,...,...,...,...,...,...,...
120440,Ghost_RUS Games,Casual,Colony On Mars,1.99,773640,"Nikita ""Ghost_RUS""",2018
120441,Sacada,Casual,LOGistICAL: South Africa,4.99,733530,Sacada,2018
120442,Laush Studio,Indie,Russian Roads,1.99,610660,Laush Dmitriy Sergeevich,2018
120443,SIXNAILS,Casual,EXIT 2 - Directions,4.99,658870,"xropi,stev3ns",2017


In [27]:
# Cuento la cantidad de nulos que quedaron por columna
print(df.isnull().sum())

genres_clean       0
app_name           2
price              0
id                 0
developer       3299
release_year       0
dtype: int64


Las columnas 'publisher' y 'developer' en su mayoria tiene valores similares. Publisher cuenta con una mamyor cantidad de valores nulos, procederemos a eliminarla.

In [26]:
df.drop(['publisher'],axis=1,inplace=True)

Cambio el nombre de la columna 'id' por 'item_id'para luego poder realizar la union de los datasets

In [30]:
df.rename(columns={'id':'item_id'},inplace=True)

Se exporta el dataset en formato parquet

In [31]:
df.to_parquet('Datasets/steams_game.parquet')